In [23]:
import torch
import pandas as pd
from ultralytics import YOLO
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import preprocess_image
import cv2
import numpy as np
import os

# Custom ActivationsAndGradients for handling tuple outputs
from pytorch_grad_cam.activations_and_gradients import ActivationsAndGradients

class ActivationsAndGradientsCustom(ActivationsAndGradients):
    def __init__(self, model, target_layers, reshape_transform=None):
        super().__init__(model, target_layers, reshape_transform)
    
    def __call__(self, x):
        # Forward pass
        outputs = self.model(x)
        
        # Assuming the first element of the tuple is what we need
        if isinstance(outputs, tuple):
            outputs = outputs[0]
        
        self.gradients = []
        self.activations = []
        
        return outputs

# Initialize the model
model = YOLO(r'C:\Users\dunli\Documents\STSY-project-main\Training_Code\runs\detect\train16\weights\best.pt')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Get the target layer for GradCAM
target_layer = model.model.model[-1]

# Directory containing the test images
image_dir = r"C:\Users\dunli\Documents\STSY-project-main\Training Data\test\images"

# DataFrame to store results
results_df = pd.DataFrame(columns=['Image', 'Class', 'Confidence', 'GradCAM_Path'])

# Define the reshape transform function
def reshape_transform(tensor):
    # Flatten tensor for GradCAM
    return tensor[0]

# Initialize GradCAMPlusPlus with the reshape transform function
cam = GradCAMPlusPlus(model=model.model, target_layers=[target_layer], use_cuda=torch.cuda.is_available())
cam.activations_and_grads = ActivationsAndGradientsCustom(cam.model, cam.target_layers, reshape_transform)

# Iterate over all images in the directory
for filename in os.listdir(image_dir):
    image_path = os.path.join(image_dir, filename)
    
    if not os.path.isfile(image_path):
        continue

    # Read and preprocess the image
    rgb_img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if rgb_img is None:
        print(f"Could not read image {image_path}")
        continue
    
    rgb_img = np.float32(rgb_img) / 255

    input_tensor = preprocess_image(
        rgb_img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
    ).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Apply GradCAM++
    grayscale_cam = cam(input_tensor=input_tensor)[0]

    heatmap = cv2.applyColorMap(
        np.uint8(255 * grayscale_cam), cv2.COLORMAP_JET
    )
    heatmap = heatmap.astype(np.float32) / 255

    alpha = 0.3
    final_output = cv2.addWeighted(rgb_img, alpha, heatmap, 1 - alpha, 0)

    # Save the heatmap
    heatmap_path = f"gradcam_{os.path.basename(image_path)}"
    cv2.imwrite(heatmap_path, (final_output * 255).astype(np.uint8))

    # Run YOLO inference
    yolo_results = model(image_path)
    for result in yolo_results:
        for box in result.boxes:
            class_name = box.cls
            confidence = box.conf

            # Append results to DataFrame
            results_df = results_df.append({
                'Image': image_path,
                'Class': class_name,
                'Confidence': confidence,
                'GradCAM_Path': heatmap_path
            }, ignore_index=True)

# Print the results DataFrame
print(results_df)


AttributeError: 'tuple' object has no attribute 'cpu'

In [64]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image
from ultralytics import YOLO
import torch

# Replace with your actual paths and model initialization
model_weights_path = r'C:\Users\dunli\Documents\STSY-project-main\Training_Code\runs\detect\train16\weights\best.pt'
image_folder = r'C:\Users\dunli\Documents\STSY-project-main\Training Data\test\images'

# Initialize YOLO model
model = YOLO(model_weights_path)

# Resize dimensions (assuming YOLO model input size)
resize_dim = (640, 640)

# Initialize EigenCAM with the target layer
target_layers = [model.model.model[-4]]  # Assuming -4 is the desired layer for EigenCAM
cam = EigenCAM(model=model.model, target_layers=target_layers, task='od')

# Iterate over each image in the directory
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    
    # Read and preprocess the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image {image_path}")
        continue
    
    rgb_img = cv2.resize(img, resize_dim)
    img = np.float32(rgb_img) / 255.0
    
    # Convert image to tensor
    img_tensor = torch.from_numpy(img.transpose(2, 0, 1)).unsqueeze(0)  # assuming RGB image
    
    # Perform inference with YOLO
    with torch.no_grad():
        outputs = model(img_tensor)
    
    # Iterate over results (assuming outputs is a list of Results objects)
    for result in outputs.xyxy:
        # Extract bounding boxes, scores, and classes
        boxes = result.cpu().numpy()[:, :4]  # Extract bounding boxes (x1, y1, x2, y2)
        scores = result.cpu().numpy()[:, 4]  # Extract confidence scores
        classes = result.cpu().numpy()[:, 5]  # Extract predicted classes
        
        # Iterate over each detected object
        for box, score, cls in zip(boxes, scores, classes):
            # Select the highest scoring box as the prediction
            class_index = int(cls)  # Convert to int for class index
            
            # Apply EigenCAM to the predicted class
            grayscale_cam = cam(img_tensor, class_index)[0, :, :]  # Compute EigenCAM for the image
            
            # Overlay EigenCAM on the original image
            cam_image = show_cam_on_image(img, grayscale_cam.numpy(), use_rgb=True)
            
            # Display the result
            plt.imshow(cam_image)
            plt.title(f'EigenCAM: {filename} - Class {class_index}')
            plt.axis('off')
            plt.show()



0: 640x640 8 rods, 2 microcolonys, 28.0ms
Speed: 1.1ms preprocess, 28.0ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)


AttributeError: 'list' object has no attribute 'xyxy'